In [1]:
import pandas as pd

In [2]:
import loader as load
ge_phylum_stage = load.loadGEOverlappingTCMA("phylum", includeStage=True)


In [3]:
import processor as pr
stad_ge_phylum_stage = ge_phylum_stage[ge_phylum_stage["project"]=="STAD"]
# print(stad_ge_phylum_stage["stage"].values)
stage_dictionary = {
    "I":0,
    "IA":0,
    "IB":0,
    "II":1,
    "IIA":1,
    "IIB":1,
    "III":2,
    "IIIA":2,
    "IIIB":2,
    "IIIC":2,
    "IV":3,
}

def convertStage(stage):
    st = str(stage)[6:]
    return stage_dictionary[st]
stad_ge_phylum_stage["stage"] = stad_ge_phylum_stage.apply(lambda row: convertStage(row.stage) if row.stage==row.stage else row.stage, axis=1)   
print(stad_ge_phylum_stage["stage"].values)
# load.attachTumorStatus(stad_ge_phylum_stage)
x, y = pr.splitData(stad_ge_phylum_stage, "stage")

[ 0.  0.  0.  2.  3.  2.  3.  1.  1.  1.  1.  3.  1.  1.  3.  2.  2. nan
  2.  2.  0. nan  1. nan nan  0.  2.  2. nan  0.  3.  2. nan  3.  2.  2.
  2.  1.  0.  3.  2.  3.  1.  3. nan  1. nan  2. nan  1.  1.  3.  2.  2.
  2.  0.  0.  2.  2.  3.  1.  0.  2.  1.  0.  2.  1. nan nan  1.  0.  2.
  1.  3.  0.  1.  0.  2.  2.  3.  1.  1. nan  1. nan  2.  2.  0.  1.  3.
  0.  1.  0.  3.  2.  1. nan  3.  1.  1.  0.  1.  1.  2.  1.  3.  3.  1.
  0.  0.  1.  0.  1.  3.  1. nan  1.  1.  1.  1.  2.  1.]


C:\Users\Ameno\AppData\Local\Temp\ipykernel_17148\3993372423.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stad_ge_phylum_stage["stage"] = stad_ge_phylum_stage.apply(lambda row: convertStage(row.stage) if row.stage==row.stage else row.stage, axis=1)


In [4]:
import predictor as pred
tcma_genus = load.loadTCMA("genus")
tcma_genus_aak_ge = load.loadGEOverlappingTCMA("genus")

load.attachTumorStatus(tcma_genus)
load.attachTumorStatus(tcma_genus_aak_ge)

x, y = pr.splitData(tcma_genus_aak_ge, "tumor", "STAD")
pred.runCrossValidation(x, y)


[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
0.9344262295081968
{'0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 4}, '1': {'precision': 0.9344262295081968, 'recall': 1.0, 'f1-score': 0.9661016949152543, 'support': 57}, 'accuracy': 0.9344262295081968, 'macro avg': {'precision': 0.4672131147540984, 'recall': 0.5, 'f1-score': 0.48305084745762716, 'support': 61}, 'weighted avg': {'precision': 0.8731523783929052, 'recall': 0.9344262295081968, 'f1-score': 0.9027507641011393, 'support': 61}}
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
0.9180327868852459
{'0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 5}, '1': {'precision': 0.9180327868852459, 'recall': 1.0, 'f1-score': 0.9572649572649572, 'support': 56}, 'accuracy': 0.9180327868852459, 'macro avg': {'precision': 0.45901639344262296, 'recall': 0.5

c:\Users\Ameno\Anaconda3\envs\bacteria\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Ameno\Anaconda3\envs\bacteria\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Ameno\Anaconda3\envs\bacteria\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Amen